## Implementing a mini chatGPT from scratch

In [1]:
# load the input dataset

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
# print the number of characters in the dataset
print(len(text))

1115393


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# now let's find all the unique characters in the data
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(' '.join(chars))
print(vocab_size)


   ! $ & ' , - . 3 : ; ? A B C D E F G H I J K L M N O P Q R S T U V W X Y Z a b c d e f g h i j k l m n o p q r s t u v w x y z
65


In [5]:
# create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]             # encoder, take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l])    # decoder, take a list of integers, output a string

print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
# now encode the entire text as a list of integers
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [7]:
# split the data into train and validation
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
print(len(train_data))
print(len(val_data))

1003853
111540


In [9]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [10]:
import numpy as np
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in np.arange(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context}, target is {target}')

when input is tensor([18]), target is 47
when input is tensor([18, 47]), target is 56
when input is tensor([18, 47, 56]), target is 57
when input is tensor([18, 47, 56, 57]), target is 58
when input is tensor([18, 47, 56, 57, 58]), target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [19]:
torch.manual_seed(1337)
batch_size = 4      # the number of independent sequences to process in parallel
block_size = 8      # the maximum context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-------')

for b in np.arange(batch_size):
    for t in np.arange(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context}, target is {target}')

inputs:
torch.Size([4, 8])
tensor([[53, 59,  6,  1, 58, 56, 47, 40],
        [49, 43, 43, 54,  1, 47, 58,  1],
        [13, 52, 45, 43, 50, 53,  8,  0],
        [ 1, 39,  1, 46, 53, 59, 57, 43]])
targets:
torch.Size([4, 8])
tensor([[59,  6,  1, 58, 56, 47, 40, 59],
        [43, 43, 54,  1, 47, 58,  1, 58],
        [52, 45, 43, 50, 53,  8,  0, 26],
        [39,  1, 46, 53, 59, 57, 43,  0]])
-------
when input is tensor([53]), target is 59
when input is tensor([53, 59]), target is 6
when input is tensor([53, 59,  6]), target is 1
when input is tensor([53, 59,  6,  1]), target is 58
when input is tensor([53, 59,  6,  1, 58]), target is 56
when input is tensor([53, 59,  6,  1, 58, 56]), target is 47
when input is tensor([53, 59,  6,  1, 58, 56, 47]), target is 40
when input is tensor([53, 59,  6,  1, 58, 56, 47, 40]), target is 59
when input is tensor([49]), target is 43
when input is tensor([49, 43]), target is 43
when input is tensor([49, 43, 43]), target is 54
when input is tensor([49, 

In [30]:
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)  

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in np.arange(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]   # (B, C)
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)             # (B, T+1)
        return idx 
    
m =  BigramLanguageModel(vocab_size)
print(np.array(m.token_embedding_table))

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)
print(idx)
print(decode(idx[0].tolist()))

Embedding(65, 65)
torch.Size([32, 65])
tensor(4.8948, grad_fn=<NllLossBackward0>)
tensor([[ 0, 31, 56, 12, 55, 28,  7, 29, 35, 49, 58, 36, 53, 24,  4, 48, 24, 16,
         22, 45, 27, 24, 34, 64,  5, 30, 21, 53, 16, 55, 20, 42, 46, 57, 34,  4,
         60, 24, 24, 62, 39, 58, 48, 57, 41, 25, 54, 61, 24, 17, 30, 31, 28, 63,
         39, 53,  8, 55, 44, 64, 57,  3, 37, 57,  3, 64, 18,  7, 61,  6, 11, 43,
         17, 49, 64, 62, 48, 45, 15, 23, 18, 15, 46, 57,  2, 47, 35, 35,  8, 27,
         40, 64, 16, 52, 62, 13,  1, 25, 57,  3,  9]])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [31]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

/opt/anaconda3/envs/miniGPT/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
batch_size = 32
for steps in np.arange(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    # zero out gradients from previous steps
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.39288067817688


In [36]:
idx = m.generate(idx=torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)
print(decode(idx[0].tolist()))


ANGO:
He rthathine!
CEShew s serer Fofow.
Houspathe t:
Mind fit.
DUKINoceamy hun.
CKI:
Norst onre t ache bar, simed?
And me theluse Bel arind-g'sto f w m CK:
YCESI fatass mbre lious ave
Wer'dor' wod y:

Henkns ges wise we me y to elil'doug p in t her spalisusin t wndalu?Y ber lishms vekeang-lumod n odas ine a! thayayor hannd t; frat.
OLArZAUSum,
s I f pin hondecharvyouk p IVIImere we keicet gs llly ideerid t com: chavily; o fill we mes co it hitodntly d d wsue ater to ovithA herisch ave une, t e


### The mathematical trick in self-attention

In [37]:
# an example

torch.manual_seed(1337)
B, T, C = 4, 8, 2   # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [40]:
# we want x[b, t] = mean_{i<=t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in np.arange(B):
    for t in np.arange(T):
        xprev = x[b, :t+1]                   # (t, C)
        xbow[b, t] = torch.mean(xprev, axis=0) # (C,)

In [54]:
# we want x[b, t] = mean_{i<=t} x[b, i]
wei = torch.tril(torch.ones(T, T))
wei = wei / torch.sum(wei, axis=1, keepdim=True)
xbow2 = wei @ x         # (B, T, T) @ (B, T, C) -----> (B, T, C)

print(torch.allclose(xbow, xbow2))

True


In [42]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [51]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [64]:
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True